# MRMR: multiresolution multi-robot visualization

Code for the visual analysis of an experiment for this paper. It is assumed that the experiments had been run with nRobot1Day-Run. 

This notebook creates the specific type of visualizations for this paper. 

In [1]:
import sys
sys.path.append("../..")
from exp_run_config import Config
Config.PROJECTNAME = "WaterBerryFarms"
import pathlib
from pprint import pprint
import gzip as compress
import pickle

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
# import matplotlib
import numpy as np

from information_model import StoredObservationIM

import wbf_figures

In [2]:
# the experiment for the robot run
experiment = "mrmr"

# run = "mrmr-3random"
# run = "mrmr-3lawnmowers"
# run = "mrmr-1pioneer-2contractors"
# run = "mrmr-clustered-1pio-2con"
run = "mrmr-clustered-3lawnmowers"

exp = Config().get_experiment(experiment, run)
pprint(exp)

resultsfile = pathlib.Path(exp["data_dir"], "results.pickle")
if not resultsfile.exists():
    print(f"Results file does not exist:\n{resultsfile}")
    print("Run the notebook Run-1Robot1Day with the same exp/run to create it.")
    raise Exception("Nothing to do.")

# load the results file
with compress.open(resultsfile, "rb") as f:
    results = pickle.load(f)

***ExpRun**: Loading pointer config file:
	/Users/lboloni/.config/WaterBerryFarms/mainsettings.yaml
***ExpRun**: Loading machine-specific config file:
	/Users/lboloni/Google Drive/My Drive/LotziStudy/Code/PackageTracking/WaterBerryFarms/settings/WBF-config-szenes.yaml
***ExpRun**: Configuration for exp/run: mrmr/mrmr-clustered-3lawnmowers successfully loaded
Experiment:
    data_dir: /Users/lboloni/Documents/Develop/Data/WBF/mrmr/mrmr-clustered-3lawnmowers
    exp_environment: mrmr-custom-clustered
    exp_estimator: adaptive-disk
    exp_run_sys_indep_file: /Users/lboloni/Documents/Develop/Github/WBF/WaterberryFarms/experiment_configs/mrmr/mrmr-clustered-3lawnmowers.yaml
    exp_score: weighted_asymmetric
    experiment_name: mrmr
    im_resolution: 10
    name: 3 lawnmowers
    robots:
    - exp-policy: fixed-budget-lawnmower
      exp-policy-extra-parameters:
        area: '[5, 5, 95, 30]'
        budget: 1000
        policy-name: lawn-bottom
      name: lawn-1
    - exp-policy: fix

Exception: Nothing to do.

### Map with detections
Creates a graph which shows the tylcv ground truth, the robot path and the detections. 

FIXME: we will need to deal with the multiple detections and priority etc.

In [ ]:


ROBOT_COLORS = ["red", "blue", "green"]
fig, ax = plt.subplots(1,1, figsize=(3, 3))
wbf_figures.show_env_tylcv(results, ax)

custom_lines = []
labels = []

for i, robot in enumerate(results["robots"]):
    color = ROBOT_COLORS[i % len(results["robots"])]
    observations = [o[i] for o in results["observations"]]
    wbf_figures.show_individual_robot_path(results, ax, robot=robot, observations=observations, pathcolor=color, draw_robot=False)
    wbf_figures.show_individual_robot_detections(results, ax, robotno=i, detection_color=color, radius=0.5)
    # adding to the legend
    custom_lines.append(Line2D([0], [0], color=color, lw=2))
    labels.append(robot.name)


# Add both automatic and manual entries to the legend
ax.legend(handles=[*custom_lines],
          labels=labels)    

plt.savefig(pathlib.Path(exp.data_dir(), "detections-map.pdf"))

### Graph with detections

Count the detections on an agent by agent basis, and create a plot for each of the agents and a sum. 

In [ ]:
def count_detections(results, robotno, field = "TYLCV"):
    """Returns the number of detections for the specified robot, adapted from wbf_figures.show_detections"""
    obs = np.array(results["observations"])[:, robotno]
    detections = [[a[StoredObservationIM.X], a[StoredObservationIM.Y]] for a in obs if a[field][StoredObservationIM.VALUE] == 0.0]
    return len(detections)

In [ ]:
fig, ax = plt.subplots(1,1)
total = 0
for i, robot in enumerate(results["robots"]):
    detections = count_detections(results, i)
    total += detections
    br = ax.bar(robot.name, detections)
ax.bar("Total", total)
plt.savefig(pathlib.Path(exp["data_dir"], "detections-bar.pdf"))